## Lab 1: AI 여행 도우미 유스케이스 소개



### 소개

이 첫 번째 실습에서는 AI 여행 도우미 유스케이스를 소개하고, 데이터 소스, Bedrock과 오픈소스 에이전트 프레임워크 LangChain의 핵심 기능을 살펴보며, 나머지 실습에 필요한 패키지와 의존성을 설정할 것입니다.

이 유스케이스는 다음과 같은 기능을 가진 봇을 만드는 것입니다:

- `RAG`를 사용하여 사용자 선호도에 기반한 비슷한 목적지를 찾는 `나의 꿈의 여행지` 기능

- 호텔과 항공편을 포함한 `여행 예약` 기능

- 작업 실행을 위한 `도구` 활용

- 중요한 실행 과정에서 `사용자 확인` 요청




### 유스케이스

당신의 모든 선호도와 요구사항을 고려하여 다음 휴가 계획을 도와줄 수 있는 AI 도우미를 상상해보세요. 이것이 바로 우리의 "AI 여행 도우미"입니다 - LangGraph와 Amazon Bedrock 모델을 사용하여 구축된 최첨단 에이전트 AI 시스템입니다. 이 도우미는 여행 계획을 원활하게 만들어주며, 모든 단계에서 개인화되고 데이터 기반의 제안을 제공합니다.

이 워크샵에서는 다음과 같은 시나리오를 다룰 것입니다:

1. **Lab 2: LangGraph와 ReACT 에이전트를 위한 빌딩 블록을 포함한 에이전트 개념 소개. 이 실습에서는 '멀티턴' 대화를 가능하게 하는 메모리 개념을 소개할 것입니다. 단기 및 장기 메모리를 보여줄 것입니다:** 

에이전트 시스템을 구축하는 데 필요한 것을 이해합니다.

2. **Lab 3: 맞춤형 여행 일정을 위한 파이프라인에 RAG를 도입한 꿈의 여행지 계획. 'human-in-the-loop'와 같은 개념을 포함한 다양한 유형의 에이전트를 보여줄 것입니다. 이 실습에서는 멀티모달 개념도 소개할 것입니다** 

다음 꿈의 여행지에 대한 개인화된 추천을 받습니다. 당신의 선호도와 과거 여행 경험을 바탕으로 맞춤형 휴가 계획을 만듭니다.

3. **Lab 4: 멀티 에이전트 협업. 이전 실습의 모든 작업을 활용하고 빌딩 블록을 조립하여 멀티 에이전트 협업(multi agent collaboration)을 만들 것입니다. 비슷한 여행자 기반 추천, 호텔 및 항공편 예약을 사용할 것입니다:** 

비슷한 연령대와 지리적 위치의 여행자들의 선호도에 영향을 받은 추천을 받습니다. 하나의 도우미를 통해 항공편과 호텔 예약을 하고, 수정하고, 취소할 수 있습니다.

4. **Lab 5: [선택사항] crew.ai를 사용한 꿈의 여행지 비교:** 

하나의 도우미를 통해 항공편과 호텔 예약을 하고, 수정하고, 취소할 수 있습니다.

5. **Lab 6: RAGAS를 사용한 에이전트 평가:** 

오픈소스 RAGAS 라이브러리를 사용하여 에이전트 워크플로우를 평가하는 방법을 이해합니다.


다이어그램은 아래와 같습니다

<img src="./images/travel_agent_light.png" width="65%" alt='travel_agent_light.png'/> 

### 데이터셋

이 워크샵에서 사용할 데이터셋 소개

#### 1. 여행 이력: 

이 워크샵에서 활용할 가상의 여행 이력 데이터셋을 만들었으며, 다음과 같은 필드를 포함합니다:

- Name: 개인의 전체 이름
- Current_Location: 개인이 현재 거주하는 도시
- Age: 개인의 나이(년)
- Past_Travel_Destinations: 개인이 이전에 방문한 도시들(쉼표로 구분된 값)
- Number_of_Trips: 개인이 여행한 총 횟수
- Flight_Number: 각 여행과 관련된 항공편 번호
- Departure_City: 각 항공편의 출발 도시
- Arrival_City: 각 항공편의 도착 도시
- Flight_Date: 각 항공편의 날짜

In [ ]:
import pandas as pd

df = pd.read_csv("./data/synthetic_travel_data.csv")
df.head(10)

#### 2. 미국과 유럽 도시 데이터셋

실습에서의 검색 증강 생성(RAG) 시연을 위해, Vector DB에 수집될 PDF 데이터셋을 가상으로 만들었습니다. 다음은 `/data/us`와 `/data/europe` 폴더에 각각 정리된 샘플 PDF입니다. 데이터셋을 만들기 위해 에이전틱 구현을 사용했습니다. 제공된 Optional 노트북에서 이를 확인할 수 있습니다.

In [ ]:
from IPython.display import Image
Image(filename='./images/vegas.png') 

### Amazon Bedrock 모델 접근

[Amazon Bedrock](https://aws.amazon.com/bedrock/)은 다양한 사용 사례에 최적화된 기초 모델(FM)을 제공하여 생성형 AI 애플리케이션을 구축하기 위한 다목적성 플랫폼입니다. 모델이 도구와 상호작용하고, 결정을 내리며, 복잡한 작업을 자율적으로 수행하는 에이전트 워크플로우에서는 적절한 FM을 선택하는 것이 매우 중요합니다. 이 시연의 목적을 위해, 우리는 Anthropic Claude, Meta의 모델, 그리고 Amazon의 자체 모델에 중점을 두고 Amazon Bedrock에서 사용 가능한 기초 모델을 사용할 것입니다. [Amazon Bedrock에서 지원되는 기초 모델](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html)을 확인해보세요.

In [ ]:
import boto3

region = 'us-west-2'
bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
)

#### Amazon Bedrock의 Anthropic Claude

`langchain_aws` 패키지는 LangChain 프레임워크를 통해 Amazon Bedrock의 언어 모델과 통합하고 상호작용하기 위한 도구입니다. 다음과 같은 이점을 제공합니다:

- Amazon Bedrock 모델에 대한 간소화된 접근
- 대화형 AI 및 에이전트 워크플로우 지원

- `from langchain_aws.chat_models.bedrock:` langchain_aws 패키지의 chat_models 모듈에서 ChatBedrock 클래스를 가져옵니다.
- `ChatBedrock:` Bedrock에서 사용 가능한 다양한 기초 모델을 사용하여 대화형 AI 애플리케이션을 만들 수 있게 해주는 Amazon Bedrock의 채팅 모델과 상호작용하기 위한 클래스입니다.

자세한 내용은 [Amazon Bedrock의 Anthropic Claude](https://aws.amazon.com/bedrock/claude/)를 참조하세요.

### Converse API

<img src="./images/converse_api.png" width="60%" alt=" conver api simplifies" />

### pip 설치 오류는 무시해주세요

In [ ]:
%pip install -U --no-cache-dir  -q \
"langchain==0.3.7" \
"langchain-aws==0.2.6" \
"langchain-community==0.3.5" \
"langchain-text-splitters==0.3.2" \
"langchainhub==0.1.20" \
"langgraph==0.2.45" \
"langgraph-checkpoint==2.0.2" \
"langgraph-sdk==0.1.35" \
"langsmith==0.1.140" \
"pypdf==3.8,<4" \
"ipywidgets>=7,<8" \
"matplotlib==3.9.0" \
"faiss-cpu==1.8.0"

%pip install -U --no-cache-dir boto3
#%pip install grandalf==3.1.2

In [ ]:
from langchain_aws.chat_models.bedrock import ChatBedrock
from rich.console import Console
from rich.text import Text

# Create a console object
console = Console()

# modelId = 'anthropic.claude-3-haiku-20240307-v1:0'
modelId = 'us.amazon.nova-lite-v1:0'
llm = ChatBedrock(
    model_id=modelId,
    client=bedrock,
    beta_use_converse_api=True
)


response = llm.invoke("이번 여름에 가족들과 하와이 여행을 가려고 해요. 5일 동안 여행 일정을 만들어주세요.").content
# Format the text as a rich Text object
formatted_text = Text.from_markup(response)

# Print the formatted response using rich
console.print(formatted_text)

### 대략적인 모델 비교

이 섹션에서는 Amazon Bedrock에서 사용 가능한 여러 모델을 실험하고 우리의 관심 과제 중 하나에 대해 대략적인 평가를 실행합니다. 시장에 있는 수천 개의 모델을 모두 평가하는 것은 불가능합니다. 따라서 신뢰할 수 있는 제공업체의 모델이면서 다양한 벤치마크에서 강력한 성능을 보여준 모델들을 사전에 필터링하는 것이 일반적으로 필요합니다.

Amazon Bedrock은 Anthropic, Meta, Mistral, Cohere, AI21Labs, Stability AI, Amazon과 같은 점점 늘어나는 제공업체 목록을 지원함으로써 빠른 후보 목록을 만들 수 있게 해줍니다. 이를 통해 효과적인 모델 선택 과정을 시작할 수 있습니다.

다음으로 우리의 관심 과제에 대한 초기 모델 선택을 알리기 위해 다음 모델에 대해 대략적인 모델 평가를 수행합니다:
- Anthropic: Claude 3 Haiku
- Amazon: Nova Lite

이 워크샵에서는 특정 모델에만 접근할 수 있습니다. 이상적으로는 자신의 계정에서 여러 모델을 사용하여 이 평가를 실행할 수 있습니다.

초기 평가를 수행하기 위해, 10개의 예제로 구성된 작은 큐레이션 데이터셋을 만듭니다. 최적의 초기 예제 수는 우리 고객들이 모델에 보낼 쿼리 유형을 대략적으로 커버할 수 있을 만큼 충분히 커야 합니다. 모델 평가 과정의 이 단계는 대략적인 아이디어를 얻기 위한 것이므로 예제 수는 작을 수 있습니다.

시작하기 위해, 우리의 시나리오는 개발 시점(**when**)에 한국어(**language**)로 여행자들(**who**)이 휴가 목적지(**what**)에 대한 요약(**task**)으로 설명될 수 있습니다. 초기 질문 세트는 [examples_ko.txt](./data/examples_ko.txt)에서 찾을 수 있습니다. 우리는 관심 있는 시나리오를 구성하는 변수 중 하나 이상을 변경하여 테스트를 확장할 수 있습니다. 예를 들어, 여행자가 아닌 사람들이나 다른 언어를 사용하는 사람들이 물어보는 동등한 예제를 생성할 수 있습니다.

In [ ]:
with open("./data/examples_ko.txt", "r") as file:
    examples = file.read().splitlines()

In [ ]:
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
from rich.console import Console
from rich.text import Text

pd.set_option('display.max_colwidth', None)
# Create a console object
console = Console()


def generate_answers(
    examples: list = [],
    system_prompt: SystemMessage = None
):
    modelIds = [
        "anthropic.claude-3-haiku-20240307-v1:0",
        'us.amazon.nova-lite-v1:0'
    ]
    output = pd.DataFrame({
        'example': [],
        'Claude3Haiku': [],
        'NovaLite': []
    })
    for example in examples:
        results = [example]
        for modelId in modelIds:
            messages = [
                system_prompt if system_prompt else SystemMessage(content=""),
                HumanMessage(content=example)
            ]
            llm = ChatBedrock(
                model_id=modelId,
                client=bedrock,
                beta_use_converse_api=True
            )
            resp = llm.invoke(messages).content
            results.append(resp)
        output.loc[len(output)] = results
    return output

In [ ]:
one_example = examples[:1]
output = generate_answers(one_example)
output.head()

### 결론

이 노트북에서는 다음을 수행했습니다:
- 이 워크샵에서 다룰 실습의 유스케이스 범위
- 모든 실습에서 사용할 데이터셋
- 이 워크샵에서 사용된 Amazon bedrock 모델

다음 실습으로 진행해주세요